In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
import optuna

/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight):
    """
    Crea e restituisce l'ambiente HalfCheetah-v5 dalla libreria Gymnasium con i parametri specificati.
    """
    return gym.make("HalfCheetah-v5", 
                    reset_noise_scale=reset_noise_scale, 
                    forward_reward_weight=forward_reward_weight, 
                    ctrl_cost_weight=ctrl_cost_weight)

# Hyperparameter tuning con Optuna
def objective(trial):
    # Parametri dell'environment
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.05, 0.2)   # Default ~0.1, esploriamo tra 0.05 e 0.2
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.8, 1.2)  # Default tipico è 1, esploriamo tra 0.8 e 1.2
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.05, 0.5)  # Default ~0.1, esploriamo tra 0.05 e 0.5
    
    # Crea l'ambiente passando i parametri
    env = make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

    # Iperparametri per il modello PPO
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    n_steps = trial.suggest_int('n_steps', 1024, 8192, step=1024)  # HalfCheetah può trarre beneficio da aggiornamenti più frequenti
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])  # Ridotto rispetto ad Ant per migliorare stabilità
    gamma = trial.suggest_float('gamma', 0.98, 0.999)  # Gamma alta per ambienti continui
    gae_lambda = trial.suggest_float('gae_lambda', 0.8, 1.0)
    clip_range = trial.suggest_float('clip_range', 0.1, 0.3)
    ent_coef = trial.suggest_float('ent_coef', 0.001, 0.05)  # Meno esplorazione rispetto ad Ant

    # Crea ed allena il modello PPO
    model = PPO("MlpPolicy", env,
                learning_rate=learning_rate,
                n_steps=n_steps,
                batch_size=batch_size,
                gamma=gamma,
                gae_lambda=gae_lambda,
                clip_range=clip_range,
                ent_coef=ent_coef,
                verbose=0)
    model.learn(total_timesteps=150000)
    
    # Disabilita aggiornamenti per valutazione
    env.training = False
    env.norm_reward = False  

    # Valuta il modello su 100 episodi
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=100, deterministic=True)

    print(f'Mean reward: {mean_reward}')
    return mean_reward

# Crea uno studio Optuna e ottimizza l'obiettivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

# Stampa i migliori iperparametri trovati
print("Best hyperparameters: ", study.best_params)


[I 2025-02-13 21:43:09,671] A new study created in memory with name: no-name-57631e5d-2102-4223-81d4-cf16a95d4c50
/var/folders/d5/8f8lxk3x1gddrb19zv8xl57m0000gn/T/ipykernel_17128/2591232108.py:23: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
[I 2025-02-13 21:44:15,362] Trial 0 finished with value: -238.57114495286

Mean reward: -238.5711449528653


/var/folders/d5/8f8lxk3x1gddrb19zv8xl57m0000gn/T/ipykernel_17128/2591232108.py:23: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[I 2025-02-13 21:45:20,610] Trial 1 finished with value: -86.29464313407327 and parameters: {'reset_noise_scale': 0.16525586112514812, 'forward_reward_weight': 0.8663739366714585, 'ctrl_cost_weight': 0.20795908742728902, 'learning_rate': 2.7422762854827535e-05, 'n_steps': 2048, 'batch_size': 64, 'gamma': 0.9903076998354103, 'gae_lambda': 0.8727467189953294, 'clip_range': 0.1740375903376737, 'ent_coef': 0.010824104279006284}. Best is trial 1 with value: -86.29464313407327.


Mean reward: -86.29464313407327


[I 2025-02-13 21:46:14,805] Trial 2 finished with value: -662.2001294437225 and parameters: {'reset_noise_scale': 0.11890046769969727, 'forward_reward_weight': 0.8284846831603818, 'ctrl_cost_weight': 0.3335688080597603, 'learning_rate': 2.5750226053273152e-05, 'n_steps': 2048, 'batch_size': 128, 'gamma': 0.9868836973780677, 'gae_lambda': 0.8757250612809824, 'clip_range': 0.24615551182071763, 'ent_coef': 0.012076358199925922}. Best is trial 1 with value: -86.29464313407327.


...

Mean reward: -16.378595204359737


[I 2025-02-13 21:53:50,816] Trial 10 finished with value: -1433.2054770581944 and parameters: {'reset_noise_scale': 0.1983162366167047, 'forward_reward_weight': 0.9490660859962773, 'ctrl_cost_weight': 0.40919292981797084, 'learning_rate': 0.0009203907025176634, 'n_steps': 1024, 'batch_size': 256, 'gamma': 0.9806812645614821, 'gae_lambda': 0.991883061084188, 'clip_range': 0.13246825198255635, 'ent_coef': 0.04963165089553444}. Best is trial 6 with value: 20.01807075359393.


Mean reward: -1433.2054770581944


[W 2025-02-13 21:54:20,002] Trial 11 failed with parameters: {'reset_noise_scale': 0.1984200497901896, 'forward_reward_weight': 1.1778452833441748, 'ctrl_cost_weight': 0.1205606585575293, 'learning_rate': 0.0001295022723954524, 'n_steps': 6144, 'batch_size': 128, 'gamma': 0.99737077340007, 'gae_lambda': 0.9683809701748427, 'clip_range': 0.17967920847825247, 'ent_coef': 0.02766888647120728} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/d5/8f8lxk3x1gddrb19zv8xl57m0000gn/T/ipykernel_17128/2591232108.py", line 41, in objective
    model.learn(total_timesteps=150000)
  File "/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/pytho

KeyboardInterrupt: 